# Advanced Object Detection

Dans l'article précédent, nous avons découvert les techniques R-CNN et Fast R-CNN, qui ont tiré parti des propositions de régions pour générer des prédictions de l'emplacement des objets dans une image ainsi que des classes correspondant aux objets de l'image. De plus, nous avons appris le goulot d'étranglement de la vitesse d'inférence, qui se produit en raison de l'existence de deux modèles différents - un pour la génération de propositions de région et un autre pour la détection d'objets. Dans cet article, nous allons découvrir différentes techniques modernes, telles que Faster R-CNN, YOLO et Single-Shot Detector (SSD), qui surmontent le temps d'inférence lente en utilisant un seul modèle pour faire des prédictions à la fois pour la classe d'objet et la limite. boîte en un seul coup. Nous commencerons par en apprendre davantage sur les boîtes d'ancrage, puis nous découvrirons le fonctionnement de chacune des techniques et leur mise en œuvre pour détecter des objets dans une image.

## Components of modern object detectionalgorithms

L'inconvénient des techniques R-CNN et Fast R-CNN est qu'elles ont deux réseaux disjoints - un pour identifier les régions qui contiennent probablement un objet et l'autre pour apporter des corrections à la boîte englobante où un objet est identifié. En outre, les deux modèles nécessitent autant de propagations vers l'avant qu'il y a de propositions de régions. Les algorithmes modernes de détection d'objets se concentrent fortement sur la formation d'un seul réseau de neurones et ont la capacité de détecter tous les objets en une seule passe avant. Dans les sections suivantes, nous découvrirons les différents composants de l'algorithme de détection d'objets moderne atypique.

* Anchor boxes
* Region proposal network (RPN)
* Region of interest pooling

# Anchor boxes

Jusqu'à présent, nous avons eu des propositions de région provenant de la méthode de recherche sélective. Les boîtes d'ancrage remplacent facilement la recherche sélective - nous apprendrons comment elles remplacent les propositions de région basées sur la recherche sélective dans cette section.

Typiquement, une majorité d'objets ont une forme similaire - par exemple, dans la majorité des cas, une boîte englobante correspondant à une image d'une personne aura une hauteur supérieure à une largeur, et une boîte englobante correspondant à l'image d'un camion aura une plus grande largeur que hauteur. Ainsi, nous aurons une idée décente de la hauteur et de la largeur des objets présents dans une image avant même d'entraîner le modèle (en inspectant les vérités fondamentales des boîtes englobantes correspondant aux objets de différentes classes).

De plus, dans certaines images, les objets d'intérêt peuvent être mis à l'échelle - ce qui entraîne une hauteur et une largeur beaucoup plus petites ou beaucoup plus grandes que la moyenne - tout en conservant le rapport hauteur/largeur $\frac{height}{width}$

Une fois que nous avons une bonne idée du rapport hauteur/largeur et de la hauteur et de la largeur des objets (qui peuvent être obtenus à partir des valeurs de vérité dans l'ensemble de données) présents dans nos images, nous définissons les boîtes d'ancrage avec des hauteurs et des largeurs représentant la majorité des boîtes englobantes des objets au sein de notre ensemble de données.

En règle générale, ceci est obtenu en utilisant le regroupement des K-moyennes au-dessus des boîtes de délimitation de la vérité fondamentale des objets présents dans les images.

Maintenant que nous comprenons comment les hauteurs et les largeurs des boîtes d'ancrage sont obtenues, nous allons apprendre comment les exploiter dans le processus.

* **1** Faites glisser chaque boîte d'ancrage sur une image du haut à gauche vers le bas à droite

* **2** La boîte d'ancrage qui a une intersection élevée sur l'union (IoU) avec l'objet aura une étiquette qui mentionne qu'elle contient un objet, et les autres seront étiquetées 0

<img src='https://www.dlology.com/static/media/uploads/yolo/car-person.jpg' width=500px>

Dans l'image précédente, nous avons deux boîtes d'ancrage, l'une qui a une hauteur supérieure à la largeur et l'autre dont la largeur est supérieure à la hauteur, pour correspondre aux objets de l'image - une personne et une voiture.

Nous glissons les deux boîtes d'ancrage sur l'image et notons les emplacements où l'IoU de la boîte d'ancrage avec la vérité est la plus élevée et notons que cet emplacement particulier contient un objet tandis que le reste des emplacements ne contient pas d'objet.

En plus des deux boîtes d'ancrage précédentes, nous créerions également des boîtes d'ancrage avec des échelles variables afin de nous adapter aux différentes échelles auxquelles un objet peut être présenté dans une image. Voici un exemple de l'apparence des différentes échelles de boîtes d'ancrage : Remarque.

<img src='https://miro.medium.com/max/1400/1*IS_9HnkfDdF00nID6xxF_A.png' width=500px>

Notez que toutes les boîtes d'ancrage ont le même centre mais des proportions ou des échelles différentes. Maintenant que nous comprenons les boîtes d'ancrage, dans la section suivante, nous allons en apprendre davantage sur le RPN, qui exploite les boîtes d'ancrage pour proposer des prédictions de régions susceptibles de contenir un objet







## Region Proposal Network

Imaginez un scénario où nous avons une image de 224 x 224 x 3. De plus, disons que la boîte d'ancrage est de forme 8 x 8 pour cet exemple. Si nous avons uu stride de 8 pixels, nous récupérons 224/8 = 28 crop d'une image pour chaque ligne - essentiellement 28 * 28 = 576 crops d'une image. Nous prenons ensuite chacun de ces recadrages et passons à travers un modèle Region Proposal Network (RPN) qui indique si le recadrage contient une image. Essentiellement, un RPN suggère la probabilité qu'une culture contenant un objet.

Comparons la sortie de la recherche sélective et la sortie d'un RPN.

la recherche sélective donne une région candidate sur la base d'un ensemble de calculs au-dessus des valeurs de pixels. Cependant, un RPN génère des régions candidates en fonction des boîtes d'ancrage et des strides avec lesquelles les boîtes d'ancrage sont glissées sur l'image. Une fois que nous obtenons les régions candidates à l'aide de l'une de ces deux méthodes, nous identifions les candidats les plus susceptibles de contenir un objet.

Alors que la génération de proposition de région basée sur la recherche sélective se fait en dehors du réseau neuronal, nous pouvons construire un RPN qui fait partie du réseau de détection d'objets. En utilisant un RPN, nous sommes maintenant dans une position où nous n'avons pas à effectuer de calculs inutiles pour calculer propositions de régions en dehors du réseau. De cette façon, nous avons un modèle unique pour identifier les régions, identifier les classes d'objets dans l'image et identifier leurs emplacements de cadre de délimitation correspondants.

Ensuite, nous apprendrons comment un RPN identifie si une région candidate (un recadrage obtenu après avoir fait glisser une boîte d'ancrage) contient ou non un objet. Dans nos données d'entraînement, nous aurions la vérité correspondant aux objets. Nous prenons maintenant chaque région candidate et comparons avec les boîtes englobantes de vérité des objets dans une image pour identifier si l'IoU entre une région candidate et une boîte englobante de vérité est supérieure à un certain seuil. Si l'IoU est supérieur à un certain seuil (disons 0,5), la région candidate contient un objet, et si l'IoU est inférieur au seuil (disons 0,1), la région candidate ne contient pas d'objet et tous les candidats qui ont une IoU entre les deux seuils (0,1 - 0,5) sont ignorés pendant l'entraînement.

Une fois que nous avons formé un modèle pour prédire si la région candidate contient un objet, nous effectuons ensuite une suppression non maximale, car plusieurs régions qui se chevauchent peuvent contenir un objet.

En résumé, un RPN forme un modèle pour lui permettre d'identifier les propositions de région avec une forte probabilité de contenir un objet en effectuant les étapes suivantes.


* **1** Faites glisser des boîtes d'ancrage de différents rapports hauteur/largeur et tailles sur l'image pour récupérer les recadrages d'une image

* **2** Calculer l'IoU entre les boîtes englobantes de vérité des objets dans l'image et les crops obtenues à l'étape précédente.

* **3** Préparez l'ensemble de données d'apprentissage de manière à ce que les crops avec une IoU supérieure à un seuil contiennent un objet et que les crops avec une IoU inférieure au seuil ne contiennent pas d'objet.

* **4** Entraîner le modèle pour identifier les régions qui contiennent un objet

* **5** Effectuez une suppression non maximale pour identifier la région candidate qui a la probabilité la plus élevée de contenir un objet et éliminer les autres régions candidates qui ont un chevauchement élevé avec elle

## Classification and regression

Jusqu'à présent, nous avons appris les étapes suivantes afin d'identifier des objets et d'effectuer des décalages par rapport aux cadres de délimitation.

* **1** Identifier les régions qui contiennent des objets.
* **2** Assurez-vous que toutes les cartes de caractéristiques des régions, quelle que soit la forme des régions, sont exactement les mêmes en utilisant la pooling des régions d'intérêt (RoI) (dont nous avons appris dans l,arcticle précédent)

Les deux problèmes liés à ces étapes sont les suivants :

* **1** Les propositions de région ne correspondent pas étroitement sur l'objet (IoU>0,5 est le seuil que nous avions dans le RPN)
* **2** Nous avons identifié si la région contient un objet ou non, mais pas la classe de l'objet situé dans la région

Nous abordons ces deux problèmes dans cette section, où nous prenons la carte de caractéristiques de forme uniforme obtenue précédemment et la passons à travers un réseau. Nous nous attendons à ce que le réseau prédise la classe de l'objet contenu dans la région et également les décalages correspondant à la région pour s'assurer que la boîte englobante est aussi étroite que possible autour de l'objet dans l'image.

<img src='https://cdn-images-1.medium.com/fit/t/1600/480/0*gW7TkaFX-4cSnkiH' width=500px>




Nous prenons la sortie du pool de RoI en entrée (la forme 7 x 7 x 512), l'aplatissons et nous nous connectons à une couche dense avant de prédire deux aspects différents

* Classe d'objet dans la région
* Quantité de décalage à faire sur les boîtes englobantes prédites de la région pour maximiser l'IoU avec la vérité



Par conséquent, s'il y a 20 classes dans les données, la sortie du réseau de neurones contient un total de 25 sorties - 21 classes (y compris la classe d'arrière-plan) et les 4 décalages à appliquer à la hauteur, la largeur et deux coordonnées centrales de la boîte de délimitation .

Avec les détails de fonctionnement de chacun des composants de Faster R-CNN en place, dans la section suivante, nous allons coder la détection d'objets à l'aide de l'algorithme Faster R-CNN.